In [ ]:
# ---
# 1. SETUP AND IMPORT LIBRARIES
# ---
import pandas as pd
import numpy as np
import os

# Plotting libraries - NOW USING PLOTLY
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Core model libraries
from prophet import Prophet
from prophet.plot import plot_components_plotly # Import plotly version
from statsmodels.tsa.stattools import acf
from scipy.stats import norm, skew, kurtosis
import scipy.stats as stats # Needed for KDE plots
from pyDOE2 import lhs

# Probabilistic evaluation libraries
import scoringrules as sr

# ---
# [NEW] PLOTTING CONFIGURATION (from your LSTM code)
# ---
print("Setting up Plotly layout for 'Times New Roman'...")

# This layout ensures a white background, Times New Roman font, and high contrast.
book_chapter_layout = go.Layout(
    title_font_family="Times New Roman",
    font=dict(family="Times New Roman", size=14, color="black"),
    paper_bgcolor='white',
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=True, 
        gridcolor='lightgrey', 
        linecolor='black', 
        zerolinecolor='lightgrey',
        mirror=True
    ),
    yaxis=dict(
        showgrid=True, 
        gridcolor='lightgrey', 
        linecolor='black', 
        zerolinecolor='lightgrey',
        mirror=True
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)', 
        bordercolor='black', 
        borderwidth=1
    ),
    hovermode="x unified"
)

# High-contrast, colorblind-friendly palette
# (Blue, Orange, Green, Red, Purple, Brown, Pink, Gray)
HC_PALETTE = ['#0072B2', '#D55E00', '#009E73', '#CC79A7', '#E69F00', '#56B4E9', '#F0E442', '#999999']

print("Libraries imported and Plotly layout configured.")

# ---
# 2. DATA LOADING AND PREPROCESSING
# ---
print("Loading data from local 'data/' folder...")

# [MODIFIED] This path now looks for a local folder
file_path = 'data/energy_dataset.csv' 

# Check if the file exists before trying to load it
if not os.path.exists(file_path):
    print("\n--- FATAL ERROR ---")
    print(f"File not found at: {file_path}")
    print("Please download the 'energy_dataset.csv' from Kaggle.")
    print("Create a 'data' folder in this project's directory and place the file inside it.")
    raise FileNotFoundError(f"File not found at {file_path}. See instructions.")

try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully.")
except Exception as e:
    print(f"An error occurred loading the data: {e}")
    raise

# Convert 'time' to datetime, set as index, and focus on the price
df['time'] = pd.to_datetime(df['time'], utc=True, errors='coerce')
df = df.set_index('time')
df = df.sort_index()

# Resample to hourly ('h') to ensure a clean index, and forward-fill missing values
df_price = df[['price actual']].resample('h').mean()
df_price = df_price.ffill()
df_price = df_price.dropna() # Drop any remaining NaNs at the start

print(f"Data loaded and preprocessed. {len(df_price)} hourly observations.")

# Split the data: Train on first 3 years, Test on the final year
test_size = 24 * 365
df_train = df_price.iloc[:-test_size]
df_test = df_price.iloc[-test_size:]

print(f"Train set: {df_train.index.min()} to {df_train.index.max()}")
print(f"Test set:  {df_test.index.min()} to {df_test.index.max()}")


# ---
# 3. DETERMINISTIC FORECAST (BASE MODEL)
# ---
print("--- Training Deterministic Base Model (Prophet) ---")
df_train_prophet = df_train.reset_index().rename(columns={'time': 'ds', 'price actual': 'y'})

# FIXED: Add this line to remove the timezone from the 'ds' column
df_train_prophet['ds'] = df_train_prophet['ds'].dt.tz_localize(None)

prophet_model = Prophet(daily_seasonality=True, 
                        weekly_seasonality=True, 
                        yearly_seasonality=True)
prophet_model.fit(df_train_prophet)
print("Prophet model fitted.")

# ---
# 4. GET DETERMINISTIC FORECAST & STOCHASTIC RESIDUALS
# ---
train_forecast = prophet_model.predict(df_train_prophet)

# FIXED: Convert to NumPy array with .values to remove the misaligning index
train_residuals = (df_train['price actual'].values - train_forecast['yhat']).dropna().values

res_mu = train_residuals.mean()
res_std = train_residuals.std()
res_skew = skew(train_residuals)
res_kurt = kurtosis(train_residuals) # Fisher kurtosis (normal=0)

print(f"Residuals captured: Mean={res_mu:.2f}, StdDev={res_std:.2f}, Skew={res_skew:.2f}, Kurtosis={res_kurt:.2f}")

# Get the deterministic forecast for the *test* period
# This will be the "base" for all our scenarios
future_df = prophet_model.make_future_dataframe(periods=len(df_test), freq='h')
base_forecast = prophet_model.predict(future_df).iloc[-len(df_test):]
deterministic_forecast = base_forecast['yhat'].values

# ---
# 5. PLOT 1: DETERMINISTIC COMPONENTS & RESIDUALS (Plotly Version)
# ---
print("Plotting deterministic components (Plotly)...")
# Use the plotly-specific plotting function from Prophet
fig_components = plot_components_plotly(prophet_model, base_forecast)
fig_components.update_layout(book_chapter_layout)
fig_components.show()

print("Plotting residual diagnostics (Plotly)...")

# --- Plot 1a: Histogram of Residuals ---
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(
    x=train_residuals, 
    nbinsx=50, 
    name='Residuals',
    marker_color=HC_PALETTE[0]
))
fig_hist.update_layout(
    title=f'Histogram of Train Residuals<br>(Mean={res_mu:.2f}, Std={res_std:.2f}, Skew={res_skew:.2f}, Kurtosis={res_kurt:.2f})',
    xaxis_title='Residual Price (€/MWh)',
    yaxis_title='Frequency'
)
fig_hist.update_layout(book_chapter_layout)
fig_hist.show()

# --- Plot 1b: Autocorrelation (ACF) of Residuals ---
acf_vals = acf(train_residuals, nlags=48, fft=True)
fig_acf = go.Figure()
fig_acf.add_trace(go.Scatter(
    x=np.arange(len(acf_vals)), 
    y=acf_vals, 
    mode='lines+markers', 
    name='ACF',
    line=dict(color=HC_PALETTE[1])
))
# Add horizontal line at 0
fig_acf.add_shape(type='line', x0=0, y0=0, x1=len(acf_vals)-1, y1=0,
                  line=dict(color='black', dash='dash'))
fig_acf.update_layout(
    title='Autocorrelation (ACF) of Residuals (The Problem to Solve)',
    xaxis_title='Lag (Hours)',
    yaxis_title='ACF'
)
fig_acf.update_layout(book_chapter_layout)
fig_acf.show()

print("ACF plot shows strong autocorrelation. Simple MC/LHS will fail to capture this.")

# ---
# 6. STOCHASTIC SCENARIO GENERATION (5 MODELS)
# ---
# (This section is pure logic, no changes needed)
num_scenarios = 1000
forecast_horizon = len(df_test)
stochastic_scenarios = {} # Holds the *residual* scenarios

# --- Model 1: Standard Monte Carlo (Parametric) ---
print("1. Running Standard Monte Carlo...")
stochastic_scenarios['Monte_Carlo'] = np.random.normal(
    res_mu, res_std, size=(num_scenarios, forecast_horizon)
)

# --- Model 2: Latin Hypercube Sampling (Parametric) ---
print("2. Running Latin Hypercube Sampling (LHS)...")
uniform_samples = lhs(n=forecast_horizon, samples=num_scenarios, criterion='maximin')
stochastic_scenarios['LHS'] = norm(loc=res_mu, scale=res_std).ppf(uniform_samples)

# --- Model 3: Historical Resampling (Bootstrap) ---
print("3. Running Historical Resampling (Bootstrap)...")
stochastic_scenarios['Bootstrap'] = np.random.choice(
    train_residuals, size=(num_scenarios, forecast_horizon), replace=True
)

# --- Model 4: Markov Chain (Discrete Autoregressive) ---
print("4. Running Markov Chain...")
n_states = 20
bins, labels = pd.qcut(train_residuals, n_states, retbins=True, labels=False)
bin_centers = {i: (labels[i] + labels[i+1])/2 for i in range(n_states)}

transitions = pd.DataFrame({'from': bins[:-1], 'to': bins[1:]})
trans_matrix = pd.crosstab(transitions['from'], transitions['to'], normalize='index')
trans_matrix_values = trans_matrix.values

mc_scenarios = np.zeros((num_scenarios, forecast_horizon), dtype=float)
for i in range(num_scenarios):
    current_state = np.random.choice(n_states)
    mc_scenarios[i, 0] = bin_centers[current_state]
    for t in range(1, forecast_horizon):
        next_state = np.random.choice(n_states, p=trans_matrix_values[current_state, :])
        mc_scenarios[i, t] = bin_centers[next_state]
        current_state = next_state
stochastic_scenarios['Markov_Chain'] = mc_scenarios

# --- Model 5: Gaussian Copula (Autoregressive) ---
print("5. Running Gaussian Copula (AR-1)...")
rho = acf_vals[1]
sorted_residuals = np.sort(train_residuals)
n_residuals = len(sorted_residuals)
gc_scenarios = np.zeros((num_scenarios, forecast_horizon))
std_dev_epsilon = np.sqrt(1 - rho**2)

for i in range(num_scenarios):
    z_t_minus_1 = np.random.normal(0, 1)
    for t in range(forecast_horizon):
        epsilon = np.random.normal(0, std_dev_epsilon)
        z_t = rho * z_t_minus_1 + epsilon
        u_t = norm.cdf(z_t)
        quantile_index = int(u_t * (n_residuals - 1))
        gc_scenarios[i, t] = sorted_residuals[quantile_index]
        z_t_minus_1 = z_t
stochastic_scenarios['Gaussian_Copula'] = gc_scenarios

print("All 5 scenario models have been run.")

# ---
# 7. GENERATE FINAL SCENARIOS & EXTENSIVE EVALUATION
# ---
# (This section is pure logic, no changes needed)

# Helper function for Pinball Loss
def pinball_loss(actuals, scenarios, quantile):
    q_forecast = np.quantile(scenarios, quantile, axis=0)
    loss = np.where(actuals > q_forecast,
                    (actuals - q_forecast) * quantile,
                    (q_forecast - actuals) * (1 - quantile))
    return np.mean(loss)

# Helper function for Reliability (Calibration)
def calculate_reliability(actuals, scenarios):
    """Calculates the empirical coverage of different quantiles."""
    quantiles = np.arange(0.1, 1.0, 0.1) # 10%, 20%, ..., 90%
    nominal_coverage = quantiles
    observed_coverage = np.zeros_like(quantiles)
    
    for i, q in enumerate(quantiles):
        q_forecast = np.quantile(scenarios, q, axis=0)
        observed_coverage[i] = np.mean(actuals <= q_forecast)
        
    p10 = np.quantile(scenarios, 0.1, axis=0)
    p90 = np.quantile(scenarios, 0.9, axis=0)
    p80_coverage = np.mean((actuals >= p10) & (actuals <= p90))
    p80_reliability = p80_coverage - 0.8 # Ideal is 0.0
    
    return nominal_coverage, observed_coverage, p80_reliability

print("--- Extended Probabilistic & Statistical Evaluation ---")
evaluation_results = {}
actuals = df_test['price actual'].values
reliability_data = {}

for model_name, residuals in stochastic_scenarios.items():
    final_scenarios = deterministic_forecast + residuals
    
    crps_score = sr.crps_ensemble(actuals, final_scenarios.T).mean()
    pinball_50 = pinball_loss(actuals, final_scenarios, 0.50)
    
    nom, obs, p80_rel = calculate_reliability(actuals, final_scenarios)
    reliability_data[model_name] = (nom, obs)
    
    res_flat = residuals.flatten()
    model_mu = res_flat.mean()
    model_std = res_flat.std()
    model_skew = skew(res_flat)
    model_kurt = kurtosis(res_flat)
    
    evaluation_results[model_name] = {
        'CRPS': crps_score,
        'P80_Reliability': p80_rel,
        'Pinball_P50': pinball_50,
        'Residual_Mean': model_mu,
        'Residual_StdDev': model_std,
        'Residual_Skew': model_skew,
        'Residual_Kurtosis': model_kurt
    }

eval_df = pd.DataFrame.from_dict(evaluation_results, orient='index')
print("--- Probabilistic Metric Results (Lower is Better, except for P80_Reliability) ---")
print(eval_df[['CRPS', 'P80_Reliability', 'Pinball_P50']].to_markdown(floatfmt=".3f"))
print("\n--- Statistical 'Fairness' Results (Compare to 'Original') ---")
eval_df.loc['Original'] = {
    'Residual_Mean': res_mu, 'Residual_StdDev': res_std,
    'Residual_Skew': res_skew, 'Residual_Kurtosis': res_kurt
}
print(eval_df[['Residual_Mean', 'Residual_StdDev', 'Residual_Skew', 'Residual_Kurtosis']].to_markdown(floatfmt=".3f"))


# ---
# 8. PLOT 2: STATISTICAL & TEMPORAL COMPARISON (Plotly Version)
# ---
print("Plotting ACF comparisons (one plot per model)...")
model_names = list(stochastic_scenarios.keys())

# --- Plot 2a: ACF of REAL Residuals (Ground Truth) ---
fig_acf_real = go.Figure()
fig_acf_real.add_trace(go.Scatter(
    x=np.arange(len(acf_vals)), 
    y=acf_vals, 
    mode='lines+markers', 
    name='Real ACF',
    line=dict(color='black', width=2)
))
fig_acf_real.add_shape(type='line', x0=0, y0=0, x1=len(acf_vals)-1, y1=0,
                  line=dict(color='black', dash='dash'))
fig_acf_real.update_layout(
    title='ACF of REAL Residuals (Ground Truth)',
    xaxis_title='Lag (Hours)',
    yaxis_title='ACF'
)
fig_acf_real.update_layout(book_chapter_layout)
fig_acf_real.show()

# --- Plot 2b-2f: ACF of each model ---
for i, model_name in enumerate(model_names):
    fig_acf_model = go.Figure()
    scen_acf = acf(stochastic_scenarios[model_name][0, :], nlags=48, fft=True)
    fig_acf_model.add_trace(go.Scatter(
        x=np.arange(len(scen_acf)), 
        y=scen_acf, 
        mode='lines+markers', 
        name=model_name,
        line=dict(color=HC_PALETTE[i])
    ))
    fig_acf_model.add_shape(type='line', x0=0, y0=0, x1=len(scen_acf)-1, y1=0,
                          line=dict(color='black', dash='dash'))
    fig_acf_model.update_layout(
        title=f'ACF of {model_name} Scenario',
        xaxis_title='Lag (Hours)',
        yaxis_title='ACF'
    )
    fig_acf_model.update_layout(book_chapter_layout)
    fig_acf_model.show()

print("Note how MC, LHS, and Bootstrap ACFs are flat (zero correlation),")
print("while Markov Chain and Gaussian Copula models correctly capture the 1-step correlation.")

# ---
# 8b. PLOT 3: RELIABILITY DIAGRAMS (Plotly Version)
# ---
print("\nPlotting Reliability Diagrams (Calibration)...")
for i, model_name in enumerate(model_names):
    fig_rel = go.Figure()
    nom, obs = reliability_data[model_name]
    
    # Plot model line
    fig_rel.add_trace(go.Scatter(
        x=nom, 
        y=obs, 
        mode='lines+markers', 
        name=model_name,
        line=dict(color=HC_PALETTE[i], width=2)
    ))
    # Plot perfect calibration line
    fig_rel.add_trace(go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        mode='lines', 
        name='Perfect Calibration',
        line=dict(color='black', dash='dash')
    ))
    
    fig_rel.update_layout(
        title=f'Reliability Diagram for {model_name}',
        xaxis_title='Nominal Quantile (Forecasted Probability)',
        yaxis_title='Observed Quantile (Actual Frequency)',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1], scaleanchor="x", scaleratio=1) # This makes the plot square
    )
    fig_rel.update_layout(book_chapter_layout)
    fig_rel.show()

# ---
# 8c. PLOT 4: RESIDUAL DISTRIBUTION (Plotly KDE Version)
# ---
print("\nPlotting Overall Residual Distributions (KDE)...")
# To replicate seaborn's kdeplot, we must compute the KDE manually
fig_kde = go.Figure()

# 1. Plot Original Residuals
kde = stats.gaussian_kde(train_residuals)
x_grid = np.linspace(train_residuals.min(), train_residuals.max(), 100)
fig_kde.add_trace(go.Scatter(
    x=x_grid, 
    y=kde(x_grid), 
    mode='lines', 
    name='Original Residuals',
    line=dict(color='black', width=3, dash='dash')
))

# 2. Plot Models
for i, model_name in enumerate(model_names):
    kde_model = stats.gaussian_kde(stochastic_scenarios[model_name].flatten())
    y_model = kde_model(x_grid)
    fig_kde.add_trace(go.Scatter(
        x=x_grid, 
        y=y_model, 
        mode='lines', 
        name=model_name,
        line=dict(color=HC_PALETTE[i], width=1.5)
    ))

fig_kde.update_layout(
    title='Statistical "Fairness": Residual Distribution Comparison',
    xaxis_title='Residual Price (€/MWh)',
    yaxis_title='Density'
)
fig_kde.update_layout(book_chapter_layout)
fig_kde.show()
print("Note: Bootstrap and Gaussian Copula should match the original's shape best.")

# ---
# 8d. PLOT 5: RAMP RATE DISTRIBUTION (Plotly KDE Version)
# ---
print("\nPlotting Ramp Rate Distributions (Decision-Making)...")
fig_ramp = go.Figure()

# 1. Plot Actual Ramps
real_ramps = np.diff(actuals)
kde_real = stats.gaussian_kde(real_ramps)
ramp_min = -25
ramp_max = 25
x_grid_ramp = np.linspace(ramp_min, ramp_max, 100)

fig_ramp.add_trace(go.Scatter(
    x=x_grid_ramp, 
    y=kde_real(x_grid_ramp), 
    mode='lines', 
    name='Actual Ramps',
    line=dict(color='black', width=3, dash='dash')
))

# 2. Plot Model Ramps
for i, model_name in enumerate(model_names):
    final_scenarios = deterministic_forecast + stochastic_scenarios[model_name]
    model_ramps = np.diff(final_scenarios, axis=1).flatten()
    kde_model = stats.gaussian_kde(model_ramps)
    y_model = kde_model(x_grid_ramp)
    fig_ramp.add_trace(go.Scatter(
        x=x_grid_ramp, 
        y=y_model, 
        mode='lines', 
        name=model_name,
        line=dict(color=HC_PALETTE[i], width=1.5)
    ))

fig_ramp.update_layout(
    title='Decision-Making: Ramp Rate Distribution Comparison',
    xaxis_title='Hourly Price Change (€/MWh)',
    yaxis_title='Density',
    xaxis=dict(range=[ramp_min, ramp_max]) # Zoom in
)
fig_ramp.update_layout(book_chapter_layout)
fig_ramp.show()
print("Note: Markov Chain and Gaussian Copula should match 'Actual Ramps' better than i.i.d. models.")


# ---
# 9. PLOT 6: FAN CHARTS OF FINAL SCENARIOS (Plotly Version)
# ---
print("Plotting scenario fan charts (one plot per model)...")
plot_slice = slice(24*7*2, 24*7*3) # Plot one week of data
time_index = df_test.index[plot_slice] # Get actual timestamps
actuals_slice = actuals[plot_slice]
deterministic_slice = deterministic_forecast[plot_slice]

# --- Plot 6a: Deterministic Forecast ---
fig_det = go.Figure()
fig_det.add_trace(go.Scatter(
    x=time_index, 
    y=deterministic_slice, 
    mode='lines', 
    name='Deterministic Forecast',
    line=dict(color=HC_PALETTE[0])
))
fig_det.add_trace(go.Scatter(
    x=time_index, 
    y=actuals_slice, 
    mode='lines', 
    name='Actual Price',
    line=dict(color=HC_PALETTE[1], width=2)
))
fig_det.update_layout(
    title='Deterministic (Base) Forecast vs. Actuals',
    yaxis_title='Price (€/MWh)',
    xaxis_title='Time'
)
fig_det.update_layout(book_chapter_layout)
fig_det.show()

# --- Plot 6b-6f: Fan charts for each model ---
for i, model_name in enumerate(model_names):
    fig_fan = go.Figure()
    final_scenarios = deterministic_forecast + stochastic_scenarios[model_name]
    
    # Plot a subset of scenarios (e.g., 50) for performance
    indices = np.random.choice(num_scenarios, 50, replace=False)
    for idx in indices:
        fig_fan.add_trace(go.Scatter(
            x=time_index, 
            y=final_scenarios[idx, plot_slice], 
            mode='lines', 
            line=dict(color='gray', width=0.5),
            opacity=0.2,
            showlegend=False
        ))
    
    # Plot P10 and P90
    p10 = np.quantile(final_scenarios, 0.1, axis=0)[plot_slice]
    p90 = np.quantile(final_scenarios, 0.9, axis=0)[plot_slice]
    
    # Plot the P10-P90 range using fill='tonexty'
    fig_fan.add_trace(go.Scatter(
        x=time_index, 
        y=p90, 
        mode='lines', 
        line=dict(width=0), 
        showlegend=False
    ))
    fig_fan.add_trace(go.Scatter(
        x=time_index, 
        y=p10, 
        mode='lines', 
        line=dict(width=0), 
        fill='tonexty',
        fillcolor=f'rgba({int(HC_PALETTE[i][1:3], 16)}, {int(HC_PALETTE[i][3:5], 16)}, {int(HC_PALETTE[i][5:7], 16)}, 0.2)',
        name='P10-P90 Range'
    ))
    
    # Plot actuals
    fig_fan.add_trace(go.Scatter(
        x=time_index, 
        y=actuals_slice, 
        mode='lines', 
        name='Actual Price',
        line=dict(color=HC_PALETTE[1], width=2)
    ))
    # Plot deterministic base
    fig_fan.add_trace(go.Scatter(
        x=time_index, 
        y=deterministic_slice, 
        mode='lines', 
        name='Base Forecast',
        line=dict(color=HC_PALETTE[0], dash='dash')
    ))
    
    fig_fan.update_layout(
        title=f'Scenarios from: {model_name}',
        xaxis_title='Time',
        yaxis_title='Price (€/MWh)'
    )
    fig_fan.update_layout(book_chapter_layout)
    fig_fan.show()

print("\n--- End of Implementation ---")